In [3]:
import clickhouse_connect
import pandas as pd
import datetime
import csv

In [4]:
client = clickhouse_connect.get_client(host='localhost', username='default', password='')

In [5]:
client.query('''DROP TABLE IF EXISTS advertising_costs''')
client.command('''
    CREATE TABLE advertising_costs (
        date DateTime,
        game VARCHAR(30),
        device VARCHAR(30),
        service VARCHAR(30),
        metric_or_else UInt32,
        cost_or_else Float32
    )
    ENGINE ReplacingMergeTree
    PRIMARY KEY (date, game, device, service);
''')

In [6]:
for i in range(1, 6):
    with open(f'data/test_task_1_batch_{i}.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        data = [(datetime.datetime.strptime(row[0], '%Y-%m-%d'), row[1], row[2],row[3], row[4], row[5]) for row in csv_reader]
        client.insert('advertising_costs', data)

In [7]:
client.query('''OPTIMIZE TABLE advertising_costs FINAL;''')

In [8]:
result = client.query('''SELECT * FROM advertising_costs''')
final_df = pd.DataFrame(columns=result.column_names, data=result.result_rows)
final_df.to_csv('data/final.csv', index=False)

In [9]:
final_df = pd.read_csv('data/final.csv')
print(final_df.shape)
final_df

(384, 6)


,date,game,device,service,metric_or_else,cost_or_else
0,2024-01-01 00:00:00+03:00,farmer,amazon,facebook,100,6953.540039
1,2024-01-01 00:00:00+03:00,farmer,amazon,instagram,100,3579.409912
2,2024-01-01 00:00:00+03:00,farmer,amazon,tiktok,1300,5503.379883
3,2024-01-01 00:00:00+03:00,farmer,amazon,youtube,800,5694.830078
4,2024-01-01 00:00:00+03:00,farmer,android,facebook,500,8586.179688
...,...,...,...,...,...,...
379,2024-01-08 00:00:00+03:00,zombie,android,youtube,1000,4198.430176
380,2024-01-08 00:00:00+03:00,zombie,ios,facebook,100,1699.430054
381,2024-01-08 00:00:00+03:00,zombie,ios,instagram,800,2882.879883
382,2024-01-08 00:00:00+03:00,zombie,ios,tiktok,0,7903.100098
